In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
from sklearn.datasets import make_moons
X, y = make_moons(n_samples=200, noise=0.2, random_state=42)

In [ ]:
print(X,y)

In [ ]:
plt.figure(figsize=(8, 6))
plt.scatter(X[:, 0], X[:, 1], c=y, cmap='viridis', s=50, edgecolor='k')
plt.title("Visualization of the Moons Dataset", fontsize=14)
plt.xlabel("Feature 1")
plt.ylabel("Feature 2")
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
y = y.reshape(-1, 1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [ ]:
import numpy as np

class ANN:
    def __init__(self, n, hidden_layers, output_size):
  
        self.layers = [n] + hidden_layers + [output_size]
        self.weights = []
        self.biases = []
        
        for i in range(len(self.layers) - 1):
            fan_in = self.layers[i]
            weight = np.random.randn(self.layers[i], self.layers[i + 1]) * np.sqrt(2. / fan_in)
            self.weights.append(weight)
            self.biases.append(np.zeros((1, self.layers[i + 1])))

    def relu(self, z):
        return np.maximum(0, z)

    def relu_derivative(self, z):
        return (z > 0).astype(float)

    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def forward(self, X):
        activations = [X]
        for w, b in zip(self.weights[:-1], self.biases[:-1]):
            z = np.dot(activations[-1], w) + b
            a = self.relu(z)
            activations.append(a)
        
        z = np.dot(activations[-1], self.weights[-1]) + self.biases[-1]
        a = self.sigmoid(z)  # Output layer with sigmoid for binary classification
        activations.append(a)
        return activations

    def compute_loss(self, Y_pred, Y_true):
        m = Y_true.shape[0]
        loss = -np.sum(Y_true * np.log(Y_pred + 1e-8) + (1 - Y_true) * np.log(1 - Y_pred + 1e-8)) / m
        return loss

    def backward(self, activations, Y_true):
        gradients = {'dW': [], 'db': []}
        m = Y_true.shape[0]
        
        # Output layer
        dz = activations[-1] - Y_true
        dw = np.dot(activations[-2].T, dz) / m
        db = np.sum(dz, axis=0, keepdims=True) / m
        gradients['dW'].insert(0, dw)
        gradients['db'].insert(0, db)
        
        # Hidden layers
        for i in range(len(self.weights) - 2, -1, -1):
            dz = np.dot(dz, self.weights[i + 1].T) * self.relu_derivative(np.dot(activations[i], self.weights[i]) + self.biases[i])
            dw = np.dot(activations[i].T, dz) / m
            db = np.sum(dz, axis=0, keepdims=True) / m
            gradients['dW'].insert(0, dw)
            gradients['db'].insert(0, db)
        
        return gradients

    def update_parameters(self, gradients, lr):
        for i in range(len(self.weights)):
            self.weights[i] -= lr * gradients['dW'][i]
            self.biases[i] -= lr * gradients['db'][i]

    def train(self, X, Y, lr, epochs):
        for epoch in range(epochs):
            # Forward
            activations = self.forward(X)
            Y_pred = activations[-1]
            
            # Loss
            loss = self.compute_loss(Y_pred, Y)
            
            # Backward
            gradients = self.backward(activations, Y)
            
            # Update
            self.update_parameters(gradients, lr)
            
            if epoch % 10 == 0:
                print(f"Epoch {epoch}, Loss: {loss:.4f}")

    def predict(self, X):
        activations = self.forward(X)
        return (activations[-1] > 0.5).astype(int)


In [ ]:
ann = ANN(n=2, hidden_layers=[], output_size=1)
ann.train(X_train, y_train, lr=0.1, epochs=300)

# Evaluate the ANN
y_pred = ann.predict(X_test)
accuracy = np.mean(y_pred == y_test) * 100
print(f"Accuracy: {accuracy:.2f}%")

In [ ]:
ann = ANN(n=2, hidden_layers=[4], output_size=1)
ann.train(X_train, y_train, lr=0.1, epochs=300)

# Evaluate the ANN
y_pred = ann.predict(X_test)
accuracy = np.mean(y_pred == y_test) * 100
print(f"Accuracy: {accuracy:.2f}%")

In [ ]:
ann = ANN(n=2, hidden_layers=[12,8], output_size=1)
ann.train(X_train, y_train, lr=0.1, epochs=300)

# Evaluate the ANN
y_pred = ann.predict(X_test)
accuracy = np.mean(y_pred == y_test) * 100
print(f"Accuracy: {accuracy:.2f}%")

In [ ]:
ann = ANN(n=2, hidden_layers=[30,20,10], output_size=1)
ann.train(X_train, y_train, lr=0.1, epochs=300)

# Evaluate the ANN
y_pred = ann.predict(X_test)
accuracy = np.mean(y_pred == y_test) * 100
print(f"Accuracy: {accuracy:.2f}%")

In [ ]:
ann = ANN(n=2, hidden_layers=[50,25,12,6], output_size=1)
ann.train(X_train, y_train, lr=0.1, epochs=300)

# Evaluate the ANN
y_pred = ann.predict(X_test)
accuracy = np.mean(y_pred == y_test) * 100
print(f"Accuracy: {accuracy:.2f}%")

In [ ]:
def plot_decision_boundary(model, X, y, resolution=0.01):
    """
    Plots the decision boundary of a trained ANN model.
    
    Parameters:
    - model: Trained ANN model
    - X: Feature matrix (2D)
    - y: Labels
    - resolution: Resolution of the grid for plotting
    """
    # Create a grid of points
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, resolution),
                         np.arange(y_min, y_max, resolution))
    
    # Flatten the grid and make predictions
    grid = np.c_[xx.ravel(), yy.ravel()]  # Shape: (n_points, 2)
    predictions = model.predict(grid)     # Shape: (n_points,)
    predictions = predictions.reshape(xx.shape)
    
    # Plot decision boundary
    plt.contourf(xx, yy, predictions, alpha=0.7, cmap=plt.cm.Paired)
    
    # Plot original data points
    plt.scatter(X[:, 0], X[:, 1], c=y.ravel(), edgecolor='k', cmap=plt.cm.Paired)
    plt.title("Decision Boundary")
    plt.xlabel("Feature 1")
    plt.ylabel("Feature 2")
    plt.show()



In [ ]:
 plot_decision_boundary(ann, X_test, y_test)